**NYPD Patrol Assistance System**

In [2]:

import pandas as pd
from pyspark import SparkContext 
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.ml import feature
from pyspark.ml import classification
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

**Data Source **

The data can be obtained from the following Website:

data.ny.gov in the name of "Motor Vehicle Crashes - Case Information: Three Year Window"


**Versions of Software**


The version of the software is Python 3.0 and Apache Spark 2.2.0

In [4]:
#Loading data onto databricks as a spark dataframe
myData = sqlContext.read.format("csv").option("header", "true").load("/FileStore/tables/Data.csv")
type(myData)

Out[ 6 ]: pyspark.sql.dataframe.DataFrame

In [5]:
#Displaying data as a table
myData.show(n=2)

+----+--------------------+-----+--------+-----------+-------------+-------------------+------------+-------------------------+-----------+------------------+------------------+----------------------+-----------------------+-----------------------------+---------------------------+--------------------+---------------------------+
Year| Crash Descriptor| Time| Date|Day of Week|Police Report|Lighting Conditions|Municipality|Collision Type Descriptor|County Name| Road Descriptor|Weather Conditions|Traffic Control Device|Road Surface Conditions|DOT Reference Marker Location|Pedestrian Bicyclist Action| Event Descriptor|Number of Vehicles Involved|
+----+--------------------+-----+--------+-----------+-------------+-------------------+------------+-------------------------+-----------+------------------+------------------+----------------------+-----------------------+-----------------------------+---------------------------+--------------------+---------------------------+
2014|Property Damage A...| 8:30|1/3/2014| Friday| Y| Daylight| WATERTOWN| OVERTAKING| JEFFERSON|Straight and Level| Clear| None| Snow/Ice| null| Not Applicable|Other Motor Vehic...| 2|
2014| Fatal Accident|14:48|1/3/2014| Friday| Y| Daylight| WATERTOWN| REAR END| JEFFERSON|Straight and Level| Cloudy| Traffic Signal| Snow/Ice| 3 73022024| Not Applicable|Other Motor Vehic...| 2|
+----+--------------------+-----+--------+-----------+-------------+-------------------+------------+-------------------------+-----------+------------------+------------------+----------------------+-----------------------+-----------------------------+---------------------------+--------------------+---------------------------+
only showing top 2 rows

**Displaying the dimension of the data**

In [7]:
#print((myData.count(), len(df.columns)))
print 'Number of rows in the dataframe : ', myData.count()
print 'Number of columns in the dataframe :' , len(myData.columns)
#The predicted variable is Crash Descriptor - it has two values - Fatal Accident and Property Damage Accident
#Displaying the breakdown of the Crash Descriptor categor variable
print 'Breakdwon of Crash Descriptor: '
myData.groupBy('Crash Descriptor').count().show()

Number of rows in the dataframe : 895916
Number of columns in the dataframe : 18
Breakdwon of Crash Descriptor: 
+--------------------+------+
 Crash Descriptor| count|
+--------------------+------+
Property Damage A...|538018|
 Fatal Accident|357898|
+--------------------+------+

In [8]:
#Displaying the column names
myData.columns

Out[ 6 ]: 
['Year',
 'Crash Descriptor',
 'Time',
 'Date',
 'Day of Week',
 'Police Report',
 'Lighting Conditions',
 'Municipality',
 'Collision Type Descriptor',
 'County Name',
 'Road Descriptor',
 'Weather Conditions',
 'Traffic Control Device',
 'Road Surface Conditions',
 'DOT Reference Marker Location',
 'Pedestrian Bicyclist Action',
 'Event Descriptor',
 'Number of Vehicles Involved']

In [9]:
type(myData)

Out[ 7 ]: pyspark.sql.dataframe.DataFrame

In [10]:
#Converting the dataframe to a pandas dataframe to manipulate the variables
Data_Pandas = myData.toPandas()


In [11]:
#Extracting the time column from the dataframe
Time = Data_Pandas['Time']
#Extracting hour information from time
Time = Time.map(lambda x:x.split(':'))
Time = Time.map(lambda x: x[0])

In [12]:
#Extracted the hour alone after eliminating the minutes
Time

Out[ 17 ]: 
0 8
1 14
2 11
3 7
4 15
5 17
6 15
7 11
8 15
9 8
10 16
11 14
12 10
13 8
14 10
15 16
16 15
17 15
18 9
19 17
20 14
21 9
22 13
23 20
24 18
25 13
26 16
27 20
28 21
29 16
 ..
895886 9
895887 13
895888 8
895889 8
895890 16
895891 14
895892 6
895893 22
895894 13
895895 21
895896 21
895897 0
895898 12
895899 0
895900 9
895901 9
895902 1
895903 9
895904 6
895905 15
895906 12
895907 1
895908 13
895909 16
895910 1
895911 15
895912 15
895913 19
895914 14
895915 15
Name: Time, dtype: object

In [13]:
#Extracting date column from the dataframe
Date = Data_Pandas['Date']
#Extracting month information from date
Date = Date.map(lambda x:x.split('/'))
Date  = Date.map(lambda x:x[0])

In [14]:
#Deleting date and time column from original dataframe to avoid problems while merging
del Data_Pandas['Date']
del Data_Pandas['Time']

In [15]:
#Displaying columns without date and time
Data_Pandas.columns

Out[ 21 ]: 
Index([u'Year', u'Crash Descriptor', u'Day of Week', u'Police Report',
 u'Lighting Conditions', u'Municipality', u'Collision Type Descriptor',
 u'County Name', u'Road Descriptor', u'Weather Conditions',
 u'Traffic Control Device', u'Road Surface Conditions',
 u'DOT Reference Marker Location', u'Pedestrian Bicyclist Action',
 u'Event Descriptor', u'Number of Vehicles Involved'],
 dtype='object')

In [16]:
#Joining the new date and time columns  to the dataframe
Data_Pandas = Data_Pandas.join(Time,how = "outer")
Data_Pandas = Data_Pandas.join(Date,how = "outer")
#Renaming the date column as Month
Data_Pandas = Data_Pandas.rename(columns = {'Date':'Month'})

In [17]:
#Validating the join by checking a random row value
Data_Pandas.iloc[818078]

Out[ 16 ]: 
Year 2016
Crash Descriptor Property Damage Accident
Day of Week Monday
Police Report Y
Lighting Conditions Daylight
Municipality HEMPSTEAD
Collision Type Descriptor OTHER
County Name NASSAU
Road Descriptor Straight and Level
Weather Conditions Clear
Traffic Control Device None
Road Surface Conditions Dry
DOT Reference Marker Location None
Pedestrian Bicyclist Action Not Applicable
Event Descriptor Other Motor Vehicle, Collision With
Number of Vehicles Involved 6
Time 16
Month 10
Name: 818078, dtype: object

In [18]:
#Display columns for validation
Data_Pandas.columns

Out[ 33 ]: 
Index([u'Year', u'Crash Descriptor', u'Day of Week', u'Police Report',
 u'Lighting Conditions', u'Municipality', u'Collision Type Descriptor',
 u'County Name', u'Road Descriptor', u'Weather Conditions',
 u'Traffic Control Device', u'Road Surface Conditions',
 u'DOT Reference Marker Location', u'Pedestrian Bicyclist Action',
 u'Event Descriptor', u'Number of Vehicles Involved', u'Time', u'Date'],
 dtype='object')

In [19]:
#Crash descriptor has two category of values  - Fatal Accident or Property Damage Accident
#We are trying to predict whether there is a possibility of Fatal accident occuring
#Creating a variable Fatal Accident that has a Y or N depending on the value in Crash descriptor 
Data_Pandas['Fatal Accident'] = Data_Pandas['Crash Descriptor'].apply(lambda x: 1.0 if x == 'Fatal Accident' else 0.0)

In [20]:
#Validating the join
Data_Pandas.columns

Out[ 19 ]: 
Index([u'Year', u'Crash Descriptor', u'Day of Week', u'Police Report',
 u'Lighting Conditions', u'Municipality', u'Collision Type Descriptor',
 u'County Name', u'Road Descriptor', u'Weather Conditions',
 u'Traffic Control Device', u'Road Surface Conditions',
 u'DOT Reference Marker Location', u'Pedestrian Bicyclist Action',
 u'Event Descriptor', u'Number of Vehicles Involved', u'Time', u'Month',
 u'Fatal Accident'],
 dtype='object')

In [21]:
#Checking for NULL values in the predicted variable
Data_Pandas['Fatal Accident'].isnull().values.any()

Out[ 20 ]: False

In [22]:
Data_Pandas['Fatal Accident'].describe(include = 'all')

Out[ 21 ]: 
count 895916.000000
mean 0.399477
std 0.489791
min 0.000000
25% 0.000000
50% 0.000000
75% 1.000000
max 1.000000
Name: Fatal Accident, dtype: float64

In [23]:
#Converting the pandas dataframe to a spark dataframe
Data = sqlContext.createDataFrame(Data_Pandas)


In [24]:
#Displaying the columns of spark dataframe
Data.columns

Out[ 22 ]: 
['Year',
 'Crash Descriptor',
 'Day of Week',
 'Police Report',
 'Lighting Conditions',
 'Municipality',
 'Collision Type Descriptor',
 'County Name',
 'Road Descriptor',
 'Weather Conditions',
 'Traffic Control Device',
 'Road Surface Conditions',
 'DOT Reference Marker Location',
 'Pedestrian Bicyclist Action',
 'Event Descriptor',
 'Number of Vehicles Involved',
 'Time',
 'Month',
 'Fatal Accident']

**Data Splitting**

The data is split into three parts:

**Training**: To fit the model

**Validation**: For Model Selection

**Testing**:For Testing the Model

In [26]:
#Splitting the data into test and train
training,validation,test = Data.randomSplit([0.7, 0.2,0.1], 0)

In [27]:
#Bucketizing Time variable
#Bins for Time
splits = [0,2,4,6,8,10,12,14,16,18,20,22,24]

MODEL 1: **Logistic Regression** (with elastic net Regularization)

The feature Engineering techniques used are:

**StringIndexer**

**OneHotEncoder**

**Bucketizer**(For Time)

All these techniques are pipelined and the Machine Learning models are fitted with the training data and validated on validation data

In [29]:
#Model 1 - Logistic Regression
model1 = Pipeline(stages=[feature.StringIndexer(inputCol='Month', outputCol='encoded_Month'),
                          feature.StringIndexer(inputCol = 'County Name',outputCol = 'encoded_County'),
                          feature.StringIndexer(inputCol = 'Day of Week',outputCol = 'encoded_Day'),
                          feature.StringIndexer(inputCol = 'Fatal Accident',outputCol = 'encoded_Fatalaccident'),
                          feature.StringIndexer(inputCol = 'Time',outputCol = 'encoded_Time'),
                          feature.OneHotEncoder(inputCol = 'encoded_Month',outputCol = 'final_Month'),
                          feature.OneHotEncoder(inputCol = 'encoded_Day',outputCol = 'final_Day'),
                          feature.OneHotEncoder(inputCol = 'encoded_County',outputCol = 'final_County'),
                          feature.Bucketizer(splits=splits, inputCol='encoded_Time', outputCol="bucketed_Time"),
  feature.VectorAssembler(inputCols = ['final_Month','final_Day','bucketed_Time','final_County'],outputCol = 'final_features'),
 classification.LogisticRegression(labelCol='encoded_Fatalaccident', featuresCol='final_features',regParam = 0.1,elasticNetParam = 0.001)])

In [30]:
#Fitting model1 with training data
fit1 = model1.fit(training)

In [31]:
#Using model1 to predict occurance of fatal acciden using validation data
dataframe1 = fit1.transform(validation)

In [32]:
dataframe1.schema.names

Out[ 33 ]: 
['Year',
 'Crash Descriptor',
 'Day of Week',
 'Police Report',
 'Lighting Conditions',
 'Municipality',
 'Collision Type Descriptor',
 'County Name',
 'Road Descriptor',
 'Weather Conditions',
 'Traffic Control Device',
 'Road Surface Conditions',
 'DOT Reference Marker Location',
 'Pedestrian Bicyclist Action',
 'Event Descriptor',
 'Number of Vehicles Involved',
 'Time',
 'Month',
 'Fatal Accident',
 'encoded_Month',
 'encoded_County',
 'encoded_Day',
 'encoded_Fatalaccident',
 'encoded_Time',
 'final_Month',
 'final_Day',
 'final_County',
 'bucketed_Time',
 'final_features',
 'rawPrediction',
 'probability',
 'prediction']

In [33]:
#Prediction vs Actual for model1
df1 = dataframe1.select('Prediction','Fatal Accident')

In [34]:
df1.head()

Out[ 29 ]: Row(Prediction=0.0, Fatal Accident=1.0)

In [35]:
# Print the coefficients and intercept for logistic regression with regularization
lrm = fit1.stages[-1]
lrm.coefficients

Out[ 51 ]: DenseVector([-0.1382, -0.0764, 0.0183, -0.0531, 0.0642, 0.0933, -0.1652, 0.0633, 0.0905, 0.1075, -0.092, -0.0144, -0.0056, -0.0038, -0.0017, -0.0035, 0.0079, -0.0134, -0.085, -0.0161, 0.7918, 0.9796, -0.0129, -0.0623, -0.2379, 0.7078, 0.9728, -0.2073, -0.3254, -0.3314, -0.1574, -0.1999, -0.4409, -0.3306, -0.3544, -0.336, 0.6284, -0.2382, -0.3545, -0.3803, -0.3867, -0.432, -0.3028, -0.5995, -0.5181, -0.3614, -0.3742, -0.4824, -0.6241, -0.35, -0.4665, -0.5336, -0.326, -0.6395, -0.4359, -0.3396, -0.3453, -0.6428, -0.5124, -0.4585, -0.4969, -0.4696, -0.5015, -0.6736, -0.6397, -0.4162, -0.5183, -0.4001, -0.3083, -0.6773, -0.2576, -0.4576, -0.3275, -0.4799, -0.4166, -0.3697, -0.4722, -0.6453, -0.4812, -0.5655])

Accuracy is calculated using Multiclass Classification Evaluator and it is found to be 68%

In [37]:
#Accuracy of model1
evaluator = MulticlassClassificationEvaluator(labelCol = 'encoded_Fatalaccident', predictionCol = 'prediction',metricName = 'accuracy')
accuracy1 = evaluator.evaluate(dataframe1)
#Error percentage:
print("Test Error = %g" % (1.0 - accuracy1))

Test Error = 0.316902

MODEL 2: **Random Forest**

The feature Engineering techniques used are:

**StringIndexer**

**OneHotEncoder**

**Bucketizer**(For Time)

All these techniques are pipelined and the Machine Learning models are fitted with the training data and validated on validation data

In [39]:
#Model 2
model2 = Pipeline(stages=[feature.StringIndexer(inputCol='Month', outputCol='encoded_Month'),
                          feature.StringIndexer(inputCol = 'County Name',outputCol = 'encoded_County'),
                          feature.StringIndexer(inputCol = 'Day of Week',outputCol = 'encoded_Day'),
                          feature.StringIndexer(inputCol = 'Fatal Accident',outputCol = 'encoded_Fatalaccident'),
                          feature.StringIndexer(inputCol = 'Time',outputCol = 'encoded_Time'),
                          feature.OneHotEncoder(inputCol = 'encoded_Month',outputCol = 'final_Month'),
                          feature.OneHotEncoder(inputCol = 'encoded_Day',outputCol = 'final_Day'),
                          feature.OneHotEncoder(inputCol = 'encoded_County',outputCol = 'final_County'),
                          feature.Bucketizer(splits=splits, inputCol='encoded_Time', outputCol="bucketed_Time"),
                 feature.VectorAssembler(inputCols = ['final_Month','final_Day','bucketed_Time','final_County'],outputCol = 'final_features'),
  classification.RandomForestClassifier(labelCol='encoded_Fatalaccident', featuresCol='final_features',numTrees = 100,maxDepth = 4)])

In [40]:
#Fitting model2 with training data
fit2 = model2.fit(training)

In [41]:
#Transforming model1 on to validation data
dataframe2 = fit2.transform(validation)

Accuracy is calculated using MultiClass Classification Evaluator and it comes to be 63%

In [43]:
#Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol = 'encoded_Fatalaccident', predictionCol = 'prediction',metricName = 'accuracy')
accuracy2 = evaluator.evaluate(dataframe2)
#Error percentage:
print("Test Error = %g" % (1.0 - accuracy2))
print(accuracy2)

Test Error = 0.366955
0.63304487555

MODEL 3: **Gradient Boosting**

The feature Engineering techniques used are:

**StringIndexer**

**OneHotEncoder**

**Bucketizer**(For Time)

All these techniques are pipelined and the Machine Learning models are fitted with the training data and validated on validation data

In [45]:
#Model 3
model3 = Pipeline(stages=[feature.StringIndexer(inputCol='Month', outputCol='encoded_Month'),
                          feature.StringIndexer(inputCol = 'County Name',outputCol = 'encoded_County'),
                          feature.StringIndexer(inputCol = 'Day of Week',outputCol = 'encoded_Day'),
                          feature.StringIndexer(inputCol = 'Fatal Accident',outputCol = 'encoded_Fatalaccident'),
                          feature.StringIndexer(inputCol = 'Time',outputCol = 'encoded_Time'),
                          feature.OneHotEncoder(inputCol = 'encoded_Month',outputCol = 'final_Month'),
                          feature.OneHotEncoder(inputCol = 'encoded_Day',outputCol = 'final_Day'),
                          feature.OneHotEncoder(inputCol = 'encoded_County',outputCol = 'final_County'),
                          feature.Bucketizer(splits=splits, inputCol='encoded_Time', outputCol="bucketed_Time"),
                 feature.VectorAssembler(inputCols = ['final_Month','final_Day','bucketed_Time','final_County'],outputCol = 'final_features'),
                 classification.GBTClassifier(labelCol='encoded_Fatalaccident', featuresCol='final_features',maxIter = 10)])

In [46]:
#Fitting model2 with training data
fit3 = model3.fit(training)

In [47]:
#Transforming model2 on to validation data
dataframe3 = fit3.transform(validation)

Accuracy is calculated using MultiClass Classification Evaluator and it is found to be 69%

In [49]:
#Accuracy
evaluator = MulticlassClassificationEvaluator(labelCol = 'encoded_Fatalaccident', predictionCol = 'prediction',metricName = 'accuracy')
accuracy3 = evaluator.evaluate(dataframe3)
#Error percentage:
print("Test Error = %g" % (1.0 - accuracy3))

Test Error = 0.31587

In [50]:
print('LogisticRegression: ',accuracy1)
print('RandomForest: ' ,accuracy2)
print('GradientBoosting: ' , accuracy3)

('LogisticRegression: ', 0.6830981593380535)
('RandomForest: ', 0.6330448755502737)
('GradientBoosting: ', 0.6841303583683445)

**Gradient Boosting** is applied to the final 10% test data since it gave the highest accuracy using Validation Data

In [52]:
#Model3(Gradient Boosting ) will be our final model.
#Applying Gradient Boosting to test data
dataframe_test = fit3.transform(test)

Final Accuracy for the test data was calculated using MultiClass Classification Evaluator and it was found to be **69%** and the best model is **Gradient Boosting**

In [54]:
#Accuracy of the selected model is 69%
evaluator = MulticlassClassificationEvaluator(labelCol = 'encoded_Fatalaccident', predictionCol = 'prediction',metricName = 'accuracy')
final_accuracy = evaluator.evaluate(dataframe_test)
#Error percentage:
print("Test Error = %g" % (1.0 - final_accuracy))
print 'Final accuracy of the model is :',final_accuracy

Test Error = 0.31459
Final accuracy of the model is : 0.685409759462